## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 358kB 4.0MB/s 
     |████████████████████████████████| 1.0MB 19.5MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
     |████████████████████████████████| 40kB 6.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 2.6MB 23.1MB/s 
ERROR: fastai 2.0.6 has requirement pandas>=1.1.0, but you'll have pandas 1.0.5 which is incompatible.


In [ ]:
from fastai.vision.all import *

In [ ]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

import cv2
from google.colab.patches import cv2_imshow
import imutils
from scipy.optimize import minimize_scalar

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [ ]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/Sudoku/Data/'

filename = 'train.tar.gz'
assert(os.path.exists(f'{mount_path}/{competition_path}/train.tar.gz'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# Functions

In [ ]:
def sqdif(bingray,sq,ang):
  rsq=imutils.resize(imutils.rotate_bound(sq,ang),width=imsz,height=imsz)

  return np.sum(np.abs(bingray-rsq))

In [ ]:
def derotate(img):
  sq=np.ones(img.shape)*255

  # binarize by blurring then using Otsu's method
  # blur=cv2.GaussianBlur(gray,(1,1),0)
  _,bingray = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

  rotang=minimize_scalar(lambda ang: sqdif(bingray,sq,ang),
                         bounds=[-45,45], method='Bounded').x

  if rotang<0:
    triang=np.tan(np.pi/4+rotang*np.pi/180)
    act_x=imsz/2*(1+triang)
    act_y=imsz/2*(1-triang)

    pts1 = np.float32([[0,act_y],[act_x,0],[imsz-act_x,imsz],[imsz,imsz-act_y]])

  else:
    triang=np.tan(np.pi/4-rotang*np.pi/180)
    act_x=imsz/2*(1-triang)
    act_y=imsz/2*(1+triang)

    pts1 = np.float32([[act_x,0],[imsz,imsz-act_y],[0,act_y],[imsz-act_x,imsz]])

  pts2 = np.float32([[0,0],[imsz,0],[0,imsz],[imsz,imsz]])
  M = cv2.getPerspectiveTransform(pts1,pts2)
  
  return cv2.warpPerspective(img,M,(imsz,imsz))

In [ ]:
# SUDOKU SOLVING ALGORITHM BY PETER NORVIG https://norvig.com/sudoku.html
# slightly modified by Benjamin P Isaacoff on 8/29/20
# original code (pre-modification by BPI) taken from https://github.com/Adityaojas/sudoku-solver/blob/master/norvig.py

def cross(A, B):
  "Cross product of elements in A and elements in B."
  return [a+b for a in A for b in B]
    
digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
         for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
         for s in squares)


def test():
  "A set of unit tests."
  assert len(squares) == 81
  assert len(unitlist) == 27
  assert np.all(len(units[s]) == 3 for s in squares)
  assert np.all(len(peers[s]) == 20 for s in squares)
  assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                          ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                          ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
  assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                              'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                              'A1', 'A3', 'B1', 'B3'])
  print('All tests pass.')


def parse_grid(grid):
  """Convert grid to a dict of possible values, {square: digits}, or
  return False if a contradiction is detected."""
  ## To start, every square can be any digit; then assign values from the grid.
  values = dict([(s, digits) for s in squares])
  for s,d in list(grid_values(grid).items()):
      if d in digits and not assign(values, s, d):
          return False ## (Fail if we can't assign d to square s.)
  return values

def grid_values(grid):
  "Convert grid into a dict of {square: char} with '0' or '.' for empties."
  chars = [c for c in grid if c in digits or c in '0.']
  assert len(chars) == 81
  return dict(list(zip(squares, chars)))


def assign(values, s, d):
  """Eliminate all the other values (except d) from values[s] and propagate.
  Return values, except return False if a contradiction is detected."""
  other_values = values[s].replace(d, '')
  if np.all([eliminate(values, s, d2) for d2 in other_values]):
      return values
  else:
      return False

def eliminate(values, s, d):
  """Eliminate d from values[s]; propagate when values or places <= 2.
  Return values, except return False if a contradiction is detected."""
  if d not in values[s]:
      return values ## Already eliminated
  values[s] = values[s].replace(d,'')
  ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
  if len(values[s]) == 0:
    return False ## Contradiction: removed last value
  elif len(values[s]) == 1:
      d2 = values[s]
      if not np.all([eliminate(values, s2, d2) for s2 in peers[s]]):
          return False
  ## (2) If a unit u is reduced to only one place for a value d, then put it there.
  for u in units[s]:
    dplaces = [s for s in u if d in values[s]]
    if len(dplaces) == 0:
        return False ## Contradiction: no place for this value
    elif len(dplaces) == 1:
        # d can only be in one place in unit; assign it there
          if not assign(values, dplaces[0], d):
              return False
  return values

def sudoku_display(instr):
  for rowind in range(9):
    rowstr=[]
    for colind in range(9):
      rowstr.append(str(instr[9*rowind+colind]))
      if colind==2 or colind==5: rowstr.append('|')
    print(' '.join(rowstr))

    if rowind==2 or rowind==5:
      print('---------------------')
  print(' ')
    
def solve(grid):
  return search(parse_grid(grid))


def search(values):
  "Using depth-first search and propagation, try all possible values."
  if values is False:
      return False ## Failed earlier
  if np.all([len(values[s]) == 1 for s in squares]): 
      return values ## Solved!
  ## Chose the unfilled square s with the fewest possibilities
  n,s = min([(len(values[s]), s) for s in squares if len(values[s]) > 1])
  return some([search(assign(values.copy(), s, d)) for d in values[s]])

def some(seq):
  "Return some element of seq that is true."
  for e in seq:
      if e: return e
  return False


#solve("400000805030000000000700000020000060000080400000010000000603070500200000104000000")
#solve([0, 0, 0, 0, 0, 0, 8, 1, 8, 0, 0, 0, 2, 3, 0, 0, 0, 6, 0, 0, 5, 7, 0, 0, 1, 0, 7, 0, 9, 6, 0, 0, 0, 0, 0, 9, 0, 7, 0, 4, 0, 1, 0, 0, 0, 0, 0, 8, 1, 0, 4, 0, 6, 0, 0, 2, 4, 0, 0, 8, 0, 0, 4, 5, 0, 0, 9, 3, 5, 0, 0, 0, 0, 0, 0])

#parse_grid("4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......")

In [ ]:
def extractNsolve(img,plot_res=False):
  curgrid=[]
    
  for rowind in range(9):
    for colind in range(9):
      subimg=img[(rowind*int(imsz/9)):((rowind+1)*int(imsz/9)),
                 (colind*int(imsz/9)):((colind+1)*int(imsz/9))]
      clfimg=learn.predict(np.stack((subimg,)*3, axis=-1))[0]
      
      if clfimg=='blank':
        curgrid.append('.')
      else:
        curgrid.append(clfimg)

      if plot_res:
        print(clfimg)
        cv2_imshow(subimg)
      
  curgrid=''.join(curgrid)

  try:
    solved=solve(curgrid)
    solved_grid=''.join(solved.values())
    
    return curgrid, solved_grid
  except:
    # print('Solving failed')
    return curgrid, 'sudontku'

  

# Load Data

In [ ]:
# this can take a bit (maybe 30s)
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists('/content/All_Data/'):
  os.mkdir('/content/All_Data/')
  shutil.unpack_archive(f'{mount_path}/{competition_path}/train.tar.gz','/content/All_Data/')  

In [ ]:
imgs=glob.glob('/content/All_Data/train/images/*.png')

In [ ]:
learn=load_learner(f'{mount_path}/My Drive/AI For Good - AI Blitz 3/Sudoku/Models/200831_1439_fullgennums.pkl')

In [ ]:
solutions=pd.read_csv('/content/All_Data/train/labels.csv')

In [ ]:
2+2

4

### Random image

In [ ]:
impath=np.random.choice(imgs)
print(impath)
imid=os.path.basename(impath)[:-4]
print(imid)

img=cv2.imread(impath)
cv2_imshow(img)

# convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imsz = gray.shape[0]


img=derotate(gray)

cv2_imshow(img)

curgrid, solved_grid = extractNsolve(img)
solstr=solutions.loc[int(imid),'solution']

print('Sudoku grid solved correctly: '+str(solved_grid==solstr))

sudoku_display(solstr)
if solved_grid != 'sudontku':
  sudoku_display(solved_grid)


In [ ]:
# curgrid, solved_grid = extractNsolve(img,plot_res=True)
# solstr==solutions.loc[int(imid),'solution']

In [ ]:
crctpuzz=[]
for impath in imgs:
  
  imid=os.path.basename(impath)[:-4]

  img=cv2.imread(impath)

  # convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  imsz = gray.shape[0]

  img=derotate(gray)

  curgrid, solved_grid = extractNsolve(img)
  solstr=solutions.loc[int(imid),'solution']

  # if solved_grid=='sudontku': print('^ '+imid+' ^')

  if solved_grid==solstr: 
    crctpuzz.append(1)
  else:
    crctpuzz.append(0)
  
  print(np.mean(crctpuzz))
